<a href="https://colab.research.google.com/github/cerr/pycerr-notebooks/blob/main/normal_tissue_complication_probability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

The Radiotherapy Outcomes Estimator<sup>1</sup> provides a library of validated models to predict normal tissue complication probability (NTCP) and tumor control probability (TCP), described via JSON files. This notebook demonstrates how to apply these models to a sample lung cancer treatment plan using pyCERR.   

1. Iyer, A., Apte, A. P., Bendau, E., Thor, M., Chen, I., Shin, J., ... & Jackson, A. (2023). ROE (Radiotherapy Outcomes Estimator): An open-source tool for optimizing radiotherapy prescriptions. Computer methods and programs in biomedicine, 242, 107833.

## Install pyCERR

In [ ]:
%%capture
!pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR.git@testing"

## Import required libraries/modules

In [ ]:
import os, glob, json, jsbeautifier
import cerr
import cerr.plan_container as pc
from cerr.roe import dosimetric_models

## Read sample data 

In [ ]:
def loadData(dataDir):
    """
    Function to import sample dataset to planC 
    """
    scanFile = os.path.join(dataDir, 'scan.nii')
    doseFile = os.path.join(dataDir, 'dose.nii')
    maskFiles = glob.glob(os.path.join(dataDir, 'mask*.nii'))

    planC = pc.loadNiiScan(scanFile, "CT SCAN")
    scanNum = len(planC.scan) - 1

    planC = pc.loadNiiDose(doseFile, scanNum, planC)

    for maskFile in maskFiles:
        structureName = (maskFile.split('_')[-1]).split('.')[0]
        planC = pc.loadNiiStructure(maskFile, scanNum, planC)
        planC.structure[-1].structureName = structureName

    return planC

#### Load sample lung RT plan

In [ ]:
sampleData = r'cerr\datasets\sample_ct\lung'
planC = loadData(sampleData)
planNum = 0
binWidth = 0.05

#Test assuming 2Gy x 40
inputFrxSize = 2
prescribedDose = 80; 

## Evaluate normal tissue complication probabilities

In [ ]:
# Location of pre-defined models
modelDir = r'cerr\roe\model_parameters'

### 1. Esophagitis

In [ ]:
modelFile = os.path.join(modelDir, 'Esophagitis (Huang).json')

ntcp = dosimetric_models.run(modelFile, planNum, planC, fNumIn=prescribedDose/inputFrxSize)

print('Esophagitis grade 2+ NTCP: ', ntcp)

Esophagitis grade 2+ NTCP:  0.10641109306350455


### 2. Pneumonitis

In [ ]:
modelFile = os.path.join(modelDir, 'Pneumonitis (Appelt).json')
with open(modelFile, 'r') as f:
   modelDict = json.load(f)

# Specify clinical characteristics
modelDict['parameters']['formerSmoker']['val'] = 1
modelDict['parameters']['over63yrs']['val'] = 1

#### Examine model parameters

In [ ]:
options = jsbeautifier.default_options()
options.indent_size = 4

# Display settings
print(jsbeautifier.beautify(json.dumps(modelDict), options))

{
    "name": "Pneumonitis (Appelt)",
    "type": "NTCP",
    "stdNumFractions": 35,
    "fractionCorrect": "Yes",
    "correctionType": "frxnum",
    "abRatio": 3,
    "parameters": {
        "structures": "LUNG - GTV1",
        "appeltMod": {
            "val": "Yes"
        },
        "D50_0": {
            "val": 34.4
        },
        "gamma50_0": {
            "val": 1.19
        },
        "formerSmoker": {
            "val": 1,
            "OR": 0.69,
            "desc": {
                "No": 0,
                "Yes": 1
            }
        },
        "currentSmoker": {
            "val": 0,
            "OR": 0.62,
            "desc": {
                "No": 0,
                "Yes": 1
            }
        },
        "over63yrs": {
            "val": 1,
            "OR": 1.66,
            "desc": {
                "No": 0,
                "Yes": 1
            }
        },
        "pulmonaryComorbidity": {
            "val": 0,
            "OR": 2.27,
            "desc": {


In [ ]:
#Evaluate 
ntcp = dosimetric_models.run(modelDict, planNum, planC, fNumIn=prescribedDose/inputFrxSize)
print('Pneumonitis grade 3+ NTCP: ', ntcp)

Pneumonitis grade 3+ NTCP:  0.11534966643177472
